In [4]:
import tensorflow as tf
from concurrent.futures import ThreadPoolExecutor
import time
import psutil
import GPUtil

# -----------------------------
# TensorFlow threading config
# -----------------------------
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

# -----------------------------
# Local image path
# -----------------------------
IMAGE_PATH = "image.jpeg"  # Make sure this exists

# -----------------------------
# Function to process image (CPU+GPU)
# -----------------------------
def process_image(_):
    img_bytes = tf.io.read_file(IMAGE_PATH)
    img = tf.io.decode_jpeg(img_bytes, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = tf.image.per_image_standardization(img)
    
    # Return small info to avoid pickling issues
    return True

# -----------------------------
# Check GPU usage
# -----------------------------
def get_gpu_usage():
    try:
        gpus = GPUtil.getGPUs()
        return [(gpu.id, gpu.load*100, gpu.memoryUtil*100) for gpu in gpus]
    except Exception:
        return []

# -----------------------------
# Start measurements
# -----------------------------
cpu_start = psutil.cpu_percent(interval=None)
gpu_start = get_gpu_usage()
start_time = time.time()

# -----------------------------
# Fake "multiprocessing" using threads
# -----------------------------
# Simulate 15 parallel processes
with ThreadPoolExecutor(max_workers=15) as executor:
    results = list(executor.map(process_image, range(15)))

end_time = time.time()
cpu_end = psutil.cpu_percent(interval=1)
gpu_end = get_gpu_usage()

# -----------------------------
# Print results
# -----------------------------
print("\n🔥 FAKE MULTIPROCESSING RESULTS (Windows-safe)")
print("===============================================")
print(f"Time taken: {end_time - start_time:.3f} sec")
print(f"CPU usage start → end: {cpu_start:.1f}% → {cpu_end:.1f}%")
print("GPU usage (end):")
for gpu_id, load, mem in gpu_end:
    print(f"  GPU-{gpu_id}: {load:.1f}% load, {mem:.1f}% mem")
print(f"Processed {sum(results)} / {len(results)} images successfully.")



🔥 FAKE MULTIPROCESSING RESULTS (Windows-safe)
Time taken: 0.170 sec
CPU usage start → end: 9.3% → 17.0%
GPU usage (end):
  GPU-0: 0.0% load, 0.0% mem
Processed 15 / 15 images successfully.
